<a href="https://colab.research.google.com/github/chasubeen/ESAA_8th_OB/blob/Week_10/%ED%8C%8C%EB%A8%B8%EC%99%84%209_5.%20%EC%BB%A8%ED%85%90%EC%B8%A0%20%EA%B8%B0%EB%B0%98%20%ED%95%84%ED%84%B0%EB%A7%81%20%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **데이터 로딩 및 가공**

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA 8기/OB/10주차/data/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


- 4803개의 데이터와 20개의 feature로 구성됨
  - 영화 제목, 개요, 인기도, 평점, 투표 수, 예산, 키워드 등 영화에 대한 다양한 메타 정보를 포함하고 있음

In [4]:
# 콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출하기

movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                    'popularity', 'keywords', 'overview']]

In [5]:
# 데이터가 구성된 형태 확인하기

pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


- genres 칼럼은 여러 개의 개별 장르 데이터를 가지고 있음
  - 개별 장르의 명칭은 딕셔너리의 key인 'name'으로 추출 가능


In [6]:
# genres 칼럼의 문자열을 분해하여 개별 장르를 파이썬 리스트 객체로 추출하기

from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [7]:
# 문자열을 객체로 변환

movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### **장르 콘텐츠 유사도 측정**

##### **Step**
1. genres를 문자열로 변경
2. 이를 CountVectorizer로 피처 벡터화
3. 벡터화 한 행렬 데이터 값을 코사인 유사도로 비교
  - 이를 위해 데이터 세트의 레코드 별 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체 생성
4. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화 추천

In [8]:
## 1) 장르를 문자열로 변경

from sklearn.feature_extraction.text import CountVectorizer

# CountVectorize를 적용하기 위해 공백 문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [9]:
## 2) CountVectorizer로 피처 벡터화

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [10]:
# 유사도 기준 내림차순 정렬

genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


### **장르 콘텐츠 필터링을 이용한 영화 추천**

In [11]:
## 장르 유사도애 따라 영화를 추천하는 함수

def find_sim_movie(df, sorted_ind, title_name, top_n = 10):

  # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
  title_movie = df[df['title'] == title_name]

  ## title_named를 가진 DataFrame의 index 객체를 ndarray로 반환하고
  # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
  title_index = title_movie.index.values
  similar_indexes = sorted_ind[title_index, :(top_n)]

  ## 추출된 top_n index들 출력, top_n index는 2차원 데이터임
  # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
  print(similar_indexes)
  similar_indexes = similar_indexes.reshape(-1)

  return df.iloc[similar_indexes]

In [12]:
# 영화 '대부;와 장르별로 유사한 영화 10개 추천

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [13]:
# 영화 평점 기준 오름차순 정렬 후 10개 데이터 확인

movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending = False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 평점 데이터가 왜곡되었다고 판단됨
  - 왜곡된 평점 데이터를 회피할 방법이 요구됨
- 유명한 영화 평점 사이트인 IMDB에서는 평가 횟수에 대한 가중치가 부여된 평점 방식을 활용함
```
가중 평점(Weighted Rating) = (v/(v+m) * R + (m/(v+m) * C)
```
  - v: 개별 영화에 평점을 투표한 횟수
  - m: 평점을 부여하기 위한 최소 투표 횟수
  - R: 개별 영화에 대한 평균 평점
  - C: 전체 영화에 대한 평균 평점

In [14]:
C = movies_df['vote_average'].mean() # 전체 영화의 평균 평점
m = movies_df['vote_count'].quantile(0.6) # 전체 투표 횟수에서 상위 60%에 해당되는 횟수
print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


In [15]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

# 기준 평점 -> 새로운 가중 평점
def weighted_vote_average(record):
  v = record['vote_count']
  R = record['vote_average']

  return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [16]:
# 새롭게 부여된 weighted_vote 평점이 높은 순으로 상위 10개의 영화 추출

movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [17]:
## 새롭게 정의된 평점 기준에 따른 영화 추천

def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
  title_movie = df[df['title'] == title_name]
  title_index = title_movie.index.values

  # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출
  similar_indexes = sorted_ind[title_index, :(top_n*2)]
  similar_indexes = similar_indexes.reshape(-1)

  # 기준 영화 index는 제외
  similar_indexes = similar_indexes[similar_indexes != title_index]

  # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출
  return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [18]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


- 이전의 추천 결과에 비해 더 나은 결과가 도출되었음